In [ ]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-06-29  189.080002  190.070007  188.940002  189.589996   
1   AAPL 2023-06-30  191.630005  194.479996  191.259995  193.970001   
2   AAPL 2023-07-03  193.779999  193.880005  191.759995  192.460007   
3   AAPL 2023-07-05  191.570007  192.979996  190.619995  191.330002   
4   AAPL 2023-07-06  189.839996  192.020004  189.199997  191.809998   

    Adj Close    Volume  
0  189.334320  46347300  
1  193.708420  85069600  
2  192.200470  31458200  
3  191.071976  46920300  
4  191.551331  45094300  


In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-06-29  189.080002  190.070007  188.940002  189.589996  189.334320   
       2023-06-30  191.630005  194.479996  191.259995  193.970001  193.708420   
       2023-07-03  193.779999  193.880005  191.759995  192.460007  192.200470   
       2023-07-05  191.570007  192.979996  190.619995  191.330002  191.071976   
       2023-07-06  189.839996  192.020004  189.199997  191.809998  191.551331   

                     Volume  
Ticker Date                  
AAPL   2023-06-29  46347300  
       2023-06-30  85069600  
       2023-07-03  31458200  
      

In [ ]:
import plotly.express as px
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [ ]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [ ]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
60  175.859001  179.779500
61  175.425002  179.368000
62  175.047000  178.683500
63  174.542000  177.835500
64  174.237001  177.040001

[65 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
195         NaN         NaN
196         NaN         NaN
197         NaN         NaN
198         NaN         NaN
199         NaN         NaN
..          ...         ...
255  135.802002  135.980001
256  135.140001  135.863001
257  134.536002  135.661501
258  133.950002  135.471502
259  133.414252  135.251127

[65 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
69          NaN         NaN
..          ...         ...
125  326.931000  328.75

In [ ]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

In [ ]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [ ]:
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',
                 title='Correlation between Apple and Microsoft')
fig.show()